In [2]:
import pandas as pd
import ast
import numpy as np

# Limpieza y transformación del dataset de output steam games

In [ ]:
# pasamos los archivos a un Dataframe
dfoutput=pd.read_json(r'output_steam_games.json',lines=True, encoding='MacRoman')

In [5]:
dfoutput.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None


In [6]:
#conservamos las columnas que nos van a servir para nuestros objetivos
dfjuegos=pd.DataFrame(dfoutput[['id','app_name','developer','publisher','release_date']].drop_duplicates()[1:])

In [7]:
dfjuegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32132 non-null  float64
 1   app_name      32132 non-null  object 
 2   developer     28835 non-null  object 
 3   publisher     24082 non-null  object 
 4   release_date  30067 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.5+ MB


In [10]:
#buscamos nulos en el dataframe para tratarlos
dfjuegos.isnull().sum()

id                 1
app_name           1
developer       3298
publisher       8051
release_date    2066
dtype: int64

In [11]:
#vemos un nulo en id
dfjuegos[dfjuegos.id.isnull()]

,id,app_name,developer,publisher,release_date
119271,NaN,Batman: Arkham City - Game of the Year Edition,"Rocksteady Studios,Feral Interactive (Mac)","Warner Bros. Interactive Entertainment, Feral ...",2012-09-07


In [12]:
#vemos 1 nulo en app_name
dfjuegos[dfjuegos.app_name.isnull()]

,id,app_name,developer,publisher,release_date
90890,317160.0,None,None,None,2014-08-26


In [20]:
# el nulo en app_name lo sacamos del url del juego
dfoutput.url[dfoutput.id==dfjuegos.id[dfjuegos.app_name.isnull()].iloc[0]].iloc[0]

'http://store.steampowered.com/app/317160/_/'

In [13]:
# sustituimos valores nulos de la columna desarrolladores con la columna de publihser, solo en las filas donde no hya nulos en publisher
dfjuegos.loc[(dfjuegos.developer.isnull())&(~dfjuegos.publisher.isnull()),'developer']=dfjuegos.publisher

In [14]:
#quitamos la columna publisher, ya que solo nos interesa la de desarrollador
dfjuegos.drop(columns='publisher',inplace=True)
dfjuegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32132 non-null  float64
 1   app_name      32132 non-null  object 
 2   developer     28900 non-null  object 
 3   release_date  30067 non-null  object 
dtypes: float64(1), object(3)
memory usage: 2.2+ MB


In [15]:
#sacamos el año de lanzamiento de la columna release_date y quitamos la columna release_date
dfjuegos['Año_lanzamiento']=dfjuegos.release_date.str.split('-',expand=True,n=1).drop(columns=1)
dfjuegos.drop(columns='release_date', inplace=True)

In [16]:
#renombramos los nombres de las columnas
dfjuegos.rename(columns={'Año lanzamiento':'Año_lanzamiento','app_name':'Nombre','developer':'Desarrollador','id':'Id'},inplace=True)

In [17]:
#hacemos una lista de los años que hay en la columna Año_de_lanzamiento
años=[]
for a in dfjuegos.Año_lanzamiento.sort_values().unique():
    try:
        int(a)
    except:
        continue
    else:
        años.append(a)
años[0:5]

['1970', '1975', '1980', '1981', '1982']

In [18]:
#cambiamos las filas que tienen el año en una parte de su cadena a contener solo el dato del año
for a in años:
    dfjuegos.loc[dfjuegos.Año_lanzamiento.str.contains(a,na=False),'Año_lanzamiento']=a

In [19]:
dfjuegos.Año_lanzamiento.sort_values().unique()

array(['"""Soon"""',
       '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
       '14 July', '1970', '1975', '1980', '1981', '1982', '1983', '1984',
       '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992',
       '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2021', 'Beta测试已开启',
       "C'est bientôt...                    (ou pas)", 'Coming Soon',
       'Coming Soon!', 'Coming Soon/Próximamente', 'Coming soon',
       'Coming soon..', 'Comming Soon',
       'Datachunks conflicted. Be vigiliant.', 'Demo coming soon.',
       'Demo is available now!', 'Early Access Starting Soon!',
       'Early Access soon', 'Not yet available', 'Play Beta in demo!',
       'Please wait warmly', 'Release Date TBA', 'SOON', 'SOON™', 'Soon',
       'Soon..'

In [49]:
dfjuegos[dfjuegos.Año_lanzamiento=='2017']

,Id,Nombre,Desarrollador,Año_lanzamiento,genres,id,tags
88312,670290.0,Real Pool 3D - Poolians,Poolians.com,2017,"[Casual, Free to Play, Indie, Simulation, Sports]",670290.0,"[Free to Play, Simulation, Sports, Casual, Ind..."
88313,767400.0,弹炸人2222,彼岸领域,2017,"[Action, Adventure, Casual]",767400.0,"[Action, Adventure, Casual]"
88333,775880.0,RC Plane 3 - Canyon Scenario,FrozenPepper,2017,"[Racing, Simulation, Sports]",775880.0,"[Simulation, Racing, Sports]"
88334,775881.0,RC Plane 3 - F-53B,FrozenPepper,2017,"[Racing, Simulation, Sports]",775881.0,"[Simulation, Racing, Sports]"
88345,773120.0,Christmas Santa Troubles,Unknown Indie Devs,2017,"[Adventure, Casual, Indie]",773120.0,"[Adventure, Indie, Casual]"
...,...,...,...,...,...,...,...
120374,773170.0,Close the Window!,Corradi Games,2017,[Casual],773170.0,[Casual]
120384,766460.0,Trireme Commander,Peter Turcan,2017,[Simulation],766460.0,[Simulation]
120399,759900.0,The 13th Heir - Ragnarok Chapter 2,Samuel James Swartout,2017,"[Action, Adventure, RPG]",759900.0,"[Action, Adventure, RPG, RPGMaker]"
120408,755310.0,CyberLink YouCam 7 Deluxe,CyberLink,2017,"[Utilities, Video Production]",755310.0,"[Utilities, Video Production]"


In [20]:
# se hace una lista de los datos que asiguen pendientes en la columna año de lanzamiento, y haciendo investigacion se encuentra que los juegos fueron lanzados del 2018 en adelante
datossinaño=[]
for a in dfjuegos.Año_lanzamiento.sort_values().unique():
    if a not in años:
        datossinaño.append(a)
datossinaño

['"""Soon"""',
 '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
 '14 July',
 'Beta测试已开启',
 "C'est bientôt...                    (ou pas)",
 'Coming Soon',
 'Coming Soon!',
 'Coming Soon/Próximamente',
 'Coming soon',
 'Coming soon..',
 'Comming Soon',
 'Datachunks conflicted. Be vigiliant.',
 'Demo coming soon.',
 'Demo is available now!',
 'Early Access Starting Soon!',
 'Early Access soon',
 'Not yet available',
 'Play Beta in demo!',
 'Please wait warmly',
 'Release Date TBA',
 'SOON',
 'SOON™',
 'Soon',
 'Soon..',
 'Soon™',
 'TBA',
 'TBD',
 'To Be Announced',
 'To be Announced',
 'To be announced',
 'When it is finished',
 'When it is ready',
 "When it's done",
 "When it's done!",
 'coming soon',
 'early access',
 'soon',
 '预热群52756441',
 None]

In [158]:
#falta comprobar si los videjugos que no tienen año son relevantes para las funciones y en caso que no, hare la conversion despues al año 2018 que es al año en que la mayoria se lanzaron

In [ ]:
# unimos los generos al dataframe de juegos
dfjuegos=dfjuegos.join(dfoutput[['genres','id','tags']],how='inner')


In [26]:
dfjuegos.head()

,Id,Nombre,Desarrollador,Año_lanzamiento,genres,id,tags
88310,761140.0,Lost Summoner Kitty,Kotoshiro,2018,"[Action, Casual, Indie, Simulation, Strategy]",761140.0,"[Strategy, Action, Indie, Casual, Simulation]"
88311,643980.0,Ironbound,Secret Level SRL,2018,"[Free to Play, Indie, RPG, Strategy]",643980.0,"[Free to Play, Strategy, Indie, RPG, Card Game..."
88312,670290.0,Real Pool 3D - Poolians,Poolians.com,2017,"[Casual, Free to Play, Indie, Simulation, Sports]",670290.0,"[Free to Play, Simulation, Sports, Casual, Ind..."
88313,767400.0,弹炸人2222,彼岸领域,2017,"[Action, Adventure, Casual]",767400.0,"[Action, Adventure, Casual]"
88314,773570.0,Log Challenge,None,None,None,773570.0,"[Action, Indie, Casual, Sports]"


In [34]:
# creamos una lista con los generos existentes
generos=[]
for a in dfjuegos.genres.dropna().drop_duplicates():
    for b in a:
        if b not in generos:
            generos.append(b)
generos=sorted(generos)
print(generos)

['Accounting', 'Action', 'Adventure', 'Animation &amp; Modeling', 'Audio Production', 'Casual', 'Design &amp; Illustration', 'Early Access', 'Education', 'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports', 'Strategy', 'Utilities', 'Video Production', 'Web Publishing']


In [43]:
# En la columna tags tenemos algunos valores de los generos que faltan en la columna generos
#los tomareos de ahi haciendonuna lista solo con los generos que tenenos ya en la columna generos
dfjuegos.loc[(dfjuegos.genres.isnull())&(~dfjuegos.tags.isnull()),['tags']].apply(lambda x: [a for a in x if a in generos])
dfjuegos.loc[(dfjuegos.genres.isnull())&(~dfjuegos.tags.isnull()),['tags']]
            

,tags
88314,"[Action, Indie, Casual, Sports]"
88321,[Casual]
88329,"[Early Access, Indie, VR]"
88330,"[Early Access, Action, Adventure, Indie, Casual]"
88332,"[Early Access, Strategy, Action, Indie, Casual..."
...,...
120428,"[Design & Illustration, Tutorial]"
120429,"[Design & Illustration, Tutorial]"
120430,[Movie]
120431,"[Design & Illustration, Tutorial]"


In [45]:
# una vez que tenemos en tags solo los generos que falatan de la columna genres, los sustituimos
# el resto de valores nulos los sustituimos por el valor 'N/D'
dfjuegos.loc[(dfjuegos.genres.isnull())&(~dfjuegos.tags.isnull()),['genres']]=dfjuegos.tags
dfjuegos.isnull().sum()

Id                    1
Nombre                1
Desarrollador      3233
Año_lanzamiento    2066
genres              138
id                    1
tags                162
dtype: int64

# ETl de dataset users items

In [2]:
#abirmos el rchivo de items en un dataframe
with open('australian_users_items.json',encoding='MacRoman') as archivo_json:
    contenido_json = archivo_json.readlines()
lisitems=[eval(line.strip())for line in contenido_json]
dfitems=pd.DataFrame(lisitems)


In [9]:
# Se desanida la columna de items y se pasa a un Dataframe el cual contiene la informacion de cada juego jugado por el usuario, como horas jugadas y el id del juego
lista_items=[] 
lista_items2=[]
for a in dfitems['items']:
    lista_items.append(ast.literal_eval(str(a)))
for a in range(len(lista_items)):
    for b in range(len(lista_items[a])):
        lista_items[a][b]['user_id']=lisitems[a]['user_id']
        lista_items2.append(lista_items[a][b])
        

In [25]:
dfitems=pd.DataFrame(lista_items2)
dfitems.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6,0,76561197970982479
1,20,Team Fortress Classic,0,0,76561197970982479
2,30,Day of Defeat,7,0,76561197970982479
3,40,Deathmatch Classic,0,0,76561197970982479
4,50,Half-Life: Opposing Force,0,0,76561197970982479


In [26]:
#observamos que hay filas duplicadas en los items
dfitems.duplicated().sum()

59104

In [30]:
#se procede a eliminar duplicados del dataframe de items y verificamos que se eliminaron
dfitems.drop_duplicates(inplace=True,ignore_index=True)
dfitems.duplicated().sum()

0

In [31]:
dfitems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   user_id           object
dtypes: int64(2), object(3)
memory usage: 194.3+ MB


In [32]:
# Se guarda en un csv el dataframe con los items desanidados y el id de usuario
dfitems.to_csv('items_por_usuario.csv',index=False)

In [34]:
# abrimos el archivo reviews
with open('australian_user_reviews.json',encoding='MacRoman') as archivo_json:
    contenido_json = archivo_json.readlines()
lisreviews=list(map(ast.literal_eval,contenido_json))
dfreviews=pd.DataFrame(lisreviews)

In [35]:
dfreviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [36]:
# Se desanida la columna de items y se pasa a un Dataframe el cual contiene la informacion de cada juego jugado por el usuario, como horas jugadas y el id del juego
lista_reviews=[] 
lista_reviews2=[]
for a in dfreviews['reviews']:
    lista_reviews.append(ast.literal_eval(str(a)))
for a in range(len(lista_reviews)):
    for b in range(len(lista_reviews[a])):
        lista_reviews[a][b]['user_id']=lisreviews[a]['user_id']
        lista_reviews2.append(lista_reviews[a][b])

In [37]:
dfreviews2=pd.DataFrame(lista_reviews2)
dfreviews2.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637


In [38]:
dfreviews2.to_csv('revies_por_usuario.csv',index=False)